# 데이터준비
> cpu 진행 예제

- 연관문서 positive samples
- 비연관문서 negative samples


## 환경설정
`FlagEmbedding`: 텍스트를 저차원 밀집 벡터(dense vector)로 변환해주는 임베딩(embedding) 라이브러리

In [1]:
%pip install tqdm FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.3/148.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.1 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.5-py3-none-any.whl size=233746 sha256=25f8f38a1678458f1a648e926abb7bd2964501127c043fc1f39e6d8958db91eb
  Stored in directory: /root/.cache/pip/wheels/b2/1f/f6/78f862bb80cb959cc9960b7c4e2d1f702b1bc0e79d19b5f124
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=62d0a151f35f78b853ecb83882c4d863d8f404c7be0650229160576b919cf5c4
  Stored in directory: /root/.c

## 데이터 준비

context기반으로 question/answer세트인데, 단답형 answer를 제공해서 새로 만들것이다.

### KLUE
https://github.com/KLUE-benchmark/KLUE/

https://huggingface.co/datasets/klue/klue

KLUE는 한국어 자연어 처리(NLP) 모델 평가를 위한 대표적인 벤치마크 데이터셋이다.
8가지 주요 한국어 NLP 과제(분류, 유사도, 추론, 개체명 인식, 관계 추출, 구문 분석, 기계 독해, 대화 상태 추적)를 포함하며,
실제 한국어 원문 기반, 오픈 라이선스, 표준화된 평가 기준을 제공한다.
한국어 NLP 연구와 모델 개발에 널리 활용된다.

In [2]:
# 46M
!wget https://github.com/KLUE-benchmark/KLUE/raw/refs/heads/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json

--2025-08-29 22:51:00--  https://github.com/KLUE-benchmark/KLUE/raw/refs/heads/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/KLUE-benchmark/KLUE/refs/heads/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json [following]
--2025-08-29 22:51:00--  https://raw.githubusercontent.com/KLUE-benchmark/KLUE/refs/heads/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47952737 (46M) [application/octet-stream]
Saving to: ‘klue-mrc-v1.1_train.json’

klue-mrc-v1.1_train 100%[=======

In [3]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [ ]:
# KLUE-MRC v1.1 train 파일 경로
filename = 'klue-mrc-v1.1_train.json'

# JSON 파일 로드
with open(filename, encoding='utf-8') as f:
    data = json.load(f)

# context, question 추출 리스트
contexts = []
questions = []

# 데이터셋 구조에 맞게 파싱 (KLUE-MRC는 'data' > 'paragraphs' > 'qas')
for article in data['data']:
    for para in article['paragraphs']:
        context = para['context']
        for qa in para['qas']:
            question = qa['question']
            contexts.append(context)
            questions.append(question)

# 데이터프레임 생성
df = pd.DataFrame({
    'context': contexts,
    'question': questions
})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17554 entries, 0 to 17553
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   17554 non-null  object
 1   question  17554 non-null  object
dtypes: object(2)
memory usage: 274.4+ KB


In [ ]:
# context 중복 제거
df = df.drop_duplicates(subset='context')

# question 중복 제거
df = df.drop_duplicates(subset='question')

# 인덱스 0부터 다시 부여
df = df.reset_index(drop=True)

# 저장
df.to_csv('klue_mrc_context_question.csv', index=False)

# 결과 확인
print(f"최종 데이터프레임 shape: {df.shape}")

df.info()
df.head()

최종 데이터프레임 shape: (13056, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13056 entries, 0 to 13055
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   13056 non-null  object
 1   question  13056 non-null  object
dtypes: object(2)
memory usage: 204.1+ KB


,context,question
0,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
1,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?
2,"미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에...",로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
3,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?
4,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?


In [ ]:
df['context'][0]

'올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.'

In [ ]:
df['question'][0]

'북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?'

### BGE 임베딩 모델
https://huggingface.co/BAAI/bge-m3

**BGE-M3**는 BAAI(Beijing Academy of Artificial Intelligence)에서 개발한 다기능, 다국어, 다중 그레뉼러리티(입력 길이 유연성)를 갖춘 임베딩 모델이다. 이 모델은 검색 및 임베딩 기반의 다양한 자연어처리(NLP) 작업에서 높은 성능과 범용성을 제공한다.

**주요 특징**

- **다기능(Multi-Functionality)**
  - 하나의 모델로 세 가지 검색 방식을 모두 지원한다.
    - **Dense Retrieval**: 문장을 하나의 임베딩 벡터로 변환하여 유사도 기반 검색 수행
    - **Sparse Retrieval(희소 검색, Lexical Matching)**: BM25와 유사하게 토큰별 가중치 기반 검색 지원
    - **Multi-Vector Retrieval(ColBERT 등)**: 문장을 여러 벡터로 표현하여 세밀한 검색 가능

- **다국어(Multi-Linguality)**
  - 100개 이상의 언어를 지원하며, 다양한 언어의 문서 및 쿼리에 대해 일관된 임베딩 품질을 제공한다.

- **다중 그레뉼러리티(Multi-Granularity)**
  - 짧은 문장부터 최대 8192 토큰에 이르는 긴 문서까지 입력 길이에 제약 없이 처리할 수 있다.

**활용 예시 및 파이프라인**

- **하이브리드 검색(Hybrid Retrieval) + 재정렬(Re-ranking)**
  - 임베딩 검색과 BM25(희소 검색)를 결합하여 정확도와 범용성을 높일 수 있다.
  - BGE-M3는 임베딩과 희소 검색을 동시에 지원하므로, 별도의 비용 없이 토큰 가중치(BM25 유사)를 함께 얻을 수 있다.
  - Vespa, Milvus 등에서 하이브리드 검색 파이프라인을 쉽게 구축할 수 있다.
  - 검색 결과에 대해 cross-encoder 기반의 re-ranker(예: bge-reranker, bge-reranker-v2)를 적용하면 더욱 높은 정확도를 얻을 수 있다.

**모델 스펙**

| 모델명                | 차원(Dimension) | 최대 시퀀스 길이 | 특징                                                         |
|----------------------|----------------|------------------|--------------------------------------------------------------|
| BAAI/bge-m3          | 1024           | 8192             | 다국어, dense/sparse/colbert 통합 파인튜닝                   |
| BAAI/bge-m3-unsupervised | 1024       | 8192             | 다국어, contrastive learning                                 |
| BAAI/bge-m3-retromae | -              | 8192             | xlm-roberta 기반, 긴 입력 지원                               |
| BAAI/bge-large-en-v1.5 | 1024         | 512              | 영어 전용 모델                                               |
| BAAI/bge-base-en-v1.5  | 768          | 512              | 영어 전용 모델                                               |
| BAAI/bge-small-en-v1.5 | 384          | 512              | 영어 전용 모델                                               |

**FAQ 요약**

- **Dense Retrieval**: 문장을 하나의 벡터로 변환(DPR, BGE-v1.5 등)
- **Sparse Retrieval**: 토큰별 가중치 벡터(BM25, splade 등)
- **Multi-Vector Retrieval**: 여러 벡터로 문장 표현(ColBERT 등)
- **파인튜닝**: dense, sparse, colbert 방식 모두 통합 파인튜닝 가능

**벤치마크 및 성능**

- MIRACL, MKQA 등 다양한 다국어 벤치마크에서 OpenAI 등 상용 모델을 능가하는 성능을 보임
- 긴 문서 검색(MLDR 데이터셋)에서도 우수한 성능을 기록함





In [ ]:
from FlagEmbedding import BGEM3FlagModel

# 1. BGE 모델 로딩
use_fp16 = True # GPU: True, CPU: False
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=use_fp16)

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

...

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

In [ ]:
# 임베딩 추출 함수
def get_embeddings(texts, batch_size=16):
    # BGEM3FlagModel encode()의 반환 타입 확인
    # 일반적으로 np.ndarray를 반환하지만, 혹시 dict면 아래처럼 처리
    # 일반적으로 GPU 메모리가 12GB 이상이면 batch_size 64~128도 무난히 사용 가능
    # - A100 40GB: batch_size 128~256도 무난 (모델 크기/임베딩 차원에 따라 상이)
    # - A100 80GB: batch_size 512 이상도 여유롭게 사용 가능
    # CPU만 사용한다면 batch_size=8~16이 안정적
    embeds = model.encode(texts, batch_size=batch_size)
    if isinstance(embeds, dict):
        # 'dense_vecs' 키에서 추출
        return np.array(embeds['dense_vecs'])
    return np.array(embeds)

In [ ]:
%%time
# 2. context 임베딩 전체 계산
context_list = df['context'].tolist()
context_embeds = get_embeddings(context_list, batch_size=256)  # shape: (num_context, dim)

# 3. question 임베딩 전체 계산
question_list = df['question'].tolist()
question_embeds = get_embeddings(question_list, batch_size=256)  # shape: (num_question, dim)

pre tokenize: 100%|██████████| 51/51 [00:14<00:00,  3.53it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 51/51 [00:09<00:00,  5.36it/s]

CPU times: user 4min 14s, sys: 1.29 s, total: 4min 16s
Wall time: 4min 16s


### Negative Sampling
**Retrieval 기반 LLM, RAG, IR(Task: 쿼리-문맥 매칭)**에서,

“정답(context)” 이외에 **가장 헷갈릴 만한 오답(negative)**을 골라서 모델이 더 정교하게 구분할 수 있게 학습 데이터로 추가하는 것이 목적이다.

**왜 ‘코사인 유사도 상위 negative’를 쓰는가?**

*Hard Negative Mining*

**아무 오답(context)**이나 negative로 주면, 이미 모델이 쉽게 맞출 수 있는 쉬운 negative들이 대부분이다.

모델이 헷갈릴 만한 “유사도는 높지만 실제로는 오답인” context(=hard negative)를 샘플링하면,

모델은 “정답 context와 비슷하지만 실제로는 정답이 아닌 것”을 더 정확히 구분하는 법을 배운다.

즉, 모델의 분별력을 높이고, 성능 향상에 도움됨.




In [ ]:
# @title 간단버젼
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# (1) 전체 유사도 행렬 계산
similarity_matrix = cosine_similarity(question_embeds, context_embeds)

# (2) 자기 자신 masking
np.fill_diagonal(similarity_matrix, -np.inf)

# (3) 각 row에서 상위 4개 인덱스를 "내림차순" 정렬로 바로 추출
topk = 4
topk_idx_sorted = np.argsort(similarity_matrix, axis=1)[:, -topk:][:, ::-1]  # (N, 4)

# (4) negative samples 할당
df['negative_samples'] = [
    [context_list[i] for i in row]
    for row in topk_idx_sorted
]


In [ ]:
# @title 정석 버젼
# - 데이터샘플수가 많은 경우 argpartition-argsort로 처리할 수 있다.

# question_embeds, context_embeds: shape (N, dim)
# context_list: length N

# (1) 모든 question, context의 코사인 유사도 행렬 (N, N) 계산
similarity_matrix = cosine_similarity(question_embeds, context_embeds)  # (N, N)

# (2) 자기 자신(정답 context) 유사도는 -inf로 마스킹
np.fill_diagonal(similarity_matrix, -np.inf)

# (3) 각 질문별 상위 4개 negative context 인덱스 추출 (벡터 연산)
topk_idx_matrix = np.argpartition(similarity_matrix, -4, axis=1)[:, -4:]  # (N, 4)

# (4) 유사도 내림차순 정렬
N = similarity_matrix.shape[0]
row_indices = np.arange(N)[:, None] # (N, 1)로 차원추가
topk_sim_values = similarity_matrix[row_indices, topk_idx_matrix]

sorted_index = np.argsort(topk_sim_values, axis=1)[:, ::-1]
topk_idx_sorted = topk_idx_matrix[row_indices, sorted_index]

# (5) negative samples 생성
negative_samples_list = [
    [context_list[i] for i in top4_idxs]
    for top4_idxs in topk_idx_sorted
]

df['negative_samples'] = negative_samples_list


#### 부분검증

In [ ]:
# np.fill_diagonal(a, val)
# - a: 대각선을 채울 배열 (보통 2차원)
# - val: 채울 값 (예: 0, 1, 999 등)
import numpy as np

a = np.zeros((3, 3), dtype=int)
np.fill_diagonal(a, 7)
print(a)

[[7 0 0]
 [0 7 0]
 [0 0 7]]


In [ ]:
# np.argpartition(a, kth, axis=-1)
# - a: 입력 배열
# - kth: 정렬 기준이 될 인덱스(들) (정수 또는 정수 리스트)
# - axis: 파티셔닝할 축
# 배열에서 k번째로 작은 값의 인덱스를 반환하는 함수
# 전체를 정렬하지 않고, 그 값보다 작은 애들은 앞으로, 큰 애들은 뒤로 보내어진다.
# k번째만 정확히 찾고, 앞뒤 인덱스는 정렬을 보장하지 않는다.
import numpy as np

a = np.array([7, 2, 3, 1, 5])
idx = np.argpartition(a, 2)
print(idx)         # 결과: [3 1 2 4 0]  (2번째로 작은 값 기준 파티션)
print(a[idx])

[3 1 2 4 0]
[1 2 3 5 7]


In [ ]:
# 차원추가
a = np.array([1, 2, 3])        # shape: (3,)
b = a[:, None]                 # shape: (3, 1)
print(b)

a = np.array([1, 2, 3])        # shape: (3,)
b = np.expand_dims(a, axis=1)  # shape: (3, 1)
print(b)

[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]


In [ ]:
df.head()

,context,question,negative_samples
0,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,[궤도물리학은 계절의 지속 기간이 지구의 궤도가 지점과 분점 사이의 공간을 휩쓸고 ...
1,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,"[한국인터넷진흥원(KISA, 원장 김석환)은 과학기술정보통신부(장관 최기영)와 포스..."
2,"미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에...",로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,[1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용...
3,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,[1960년에 주니치에 입단하여 같은 해 1960년 5월 7일 다이요 웨일스전에서 ...
4,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,[‘짜파구리’로 시작된 ‘나만의 레시피’ 열풍이 올해 식품시장을 주도한 것으로 나타...


In [ ]:
df.to_csv('klue_mrc_context_question_negative_samples.csv', index=False)

In [ ]:
print(f"negative_samples #: {len(df['negative_samples'][10])} 건")
df['negative_samples'][0]

negative_samples #: 4 건


['궤도물리학은 계절의 지속 기간이 지구의 궤도가 지점과 분점 사이의 공간을 휩쓸고 지나가는 면적이 클수록 길어지며, 따라서 만약 이심률이 극단적으로 커진다면 원일점 쪽에서 나타나는 계절이 오래 지속될 것이다고 예측한다. 현재 지구에서는, 지구가 근일점에 접근할수록(태양에 가까워질수록) 북반구는 가을을 지나 겨울로 향하고, 한편 남반구에서는 반대되는 계절이 나타나고 있다. 결과적으로, 북반구에서는 가을과 겨울이 봄과 여름보다 살짝 짧다. 하지만, 전 지구적으로 보았을 때는 남반구는 오히려 봄과 여름이 살짝 짧음으로서 균형이 맞는다. 2006년에는 밀란코비치 주기에 따라 북반구의 여름이 겨울보다 4.66일 더 길었고, 봄은 가을보다 2.9일 더 길었다. \n\n장축단선의 세차운동 또한 지구의 지점과 분점의 위치를 느리게 바꾸고 있다. 참고로, 이 움직임은 지구의 "궤도"를 바꾸는 것이지 지구의 자전축을 바꾸는 것이 아니다(자전축 변화는 자전축의 세차운동 참조). 다음 1만 년 동안, 북반구의 겨울은 조금씩 길어질 것이고 여름은 조금씩 짧아질 것이다. 하지만, 한 쪽이 차가워짐에 따라 반대쪽은 따뜻해지는 것처럼 어떠한 영향도 반대의 영향을 받을 것이다.',
 '2017년까지 강원 양양과 충남 태안, 경남 남해 등 12개 해안 지역거점 12곳이 휴양·체험·생태벨트로 조성된다.국토교통부는 지난해부터 동서남해안 일대에 조성 중인 ‘휴양·체험·생태벨트’에 4곳을 추가해 총 12곳을 개발한다고 25일 발표했다.국토부는 지난해 동해 망상과 영덕 고래불, 강릉 심곡, 울산 진하, 전북 고창, 전남 진도항, 전남 고흥, 경남 거제 등 8곳을 휴양·체험·생태벨트 거점으로 개발하기로 하고 사업을 진행 중이다.새로 추가된 양양에는 오색 자연휴양체험지구가 조성된다. 올해부터 2017년까지 국비와 지방비 등 300억원을 투자해 양양군 서면 오색리 35만㎡ 일대에 캠핑장 등 자연휴양 체험시설을 짓는다. 태안군 소원면 만리포해수욕장 내 31만1853㎡에는 180억원을 들여 해안도로와 탐방로